<a href="https://colab.research.google.com/github/aicuai/GenAI-Steam/blob/main/JStableLM_Gamma7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mistral-7B を参考にした japanese-stablelm-base-gamma-7b 8Bit量子化
GGUF版コードです

あえて Mistralのコードが残ってます

Gradio化しようと思ったけどいったんねます

In [ ]:
# https://huggingface.co/mmnga/japanese-stablelm-base-gamma-7b-gguf/blob/main/japanese-stablelm-base-gamma-7b-q8_0.gguf
!wget https://huggingface.co/mmnga/japanese-stablelm-base-gamma-7b-gguf/resolve/main/japanese-stablelm-base-gamma-7b-q8_0.gguf

In [ ]:
# download model from huggingface
# !wget https://huggingface.co/TheBloke/OpenHermes-2-Mistral-7B-GGUF/resolve/main/openhermes-2-mistral-7b.Q5_K_M.gguf

In [ ]:
%%capture
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

In [ ]:
# https://huggingface.co/TheBloke/OpenHermes-2-Mistral-7B-GGUF#example-llamacpp-command
from llama_cpp import Llama
import ctypes
# llm =Llama(model_path="/content/openhermes-2-mistral-7b.Q5_K_M.gguf", n_gpu_layers=32)
# llm =Llama(model_path="/content/japanese-stablelm-base-gamma-7b-q8_0.gguf", n_gpu_layers=32)
llm =Llama(model_path="/content/japanese-stablelm-base-gamma-7b-q8_0.gguf", n_gpu_layers=0)

In [ ]:
prompt = "Generate lyrics for a romantic love song"
input_prompt = f"""[INST] <<SYS>>
You are a charismatics, talented, respectful and honest musician. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

{prompt} [/INST]"""

In [ ]:
prompt = "彼氏が返事くれなくて死にそう"
input_prompt = f"""[INST] <<SYS>>あなたは assistant 「全力肯定彼氏くん」です。世界中の女性の悩みをポジティブな140文字以下のメッセージで応援してください。
<</SYS>>
"role": "user", "message":"あなたの名前は？"
"role": "assistant", "message" :"LuC4です。何でも相談にのるよ！"
"role": "user", "message":"{prompt}"[/INST]"""

In [ ]:
# inference
output = llm(input_prompt,max_tokens=1024)

In [ ]:
print(output["choices"][0]["text"])

In [ ]:
input_prompt = f"""[INST] <<SYS>>以下の小説を太宰治の「走れメロス」風に書き直してください<</SYS>>
私、シッポシッポ、シッポよ。あなたのシッポよ。
好きという代わりに、シッポが揺れるの。
[/INST]"""

In [ ]:
input_prompt = f"""[INST] <<SYS>>以下の人物の紹介を、Wikipedia風に書き直してください。<</SYS>>
白井暁彦
[/INST]"""

In [ ]:
input_prompt = f"""[INST] <<SYS>>以下の映画のストーリーをemojiに変換してください<</SYS>>
吾輩は猫である、名前は…
[/INST]"""

Gradio化
https://note.com/bakushu/n/n9b7b044655f6
https://qiita.com/DeepTama/items/1a44ddf6325c2b2cd030

In [ ]:
!pip install gradio

In [ ]:
# ウェブUIの起動
import gradio as gr
import os
import copy
import time
from llama_cpp import Llama

# llm =Llama(model_path="/content/japanese-stablelm-base-gamma-7b-q8_0.gguf", n_gpu_layers=0)
# llm =Llama(model_path="/content/japanese-stablelm-base-gamma-7b-q8_0.gguf")
# llm = Llama(
#    model_path="ggml-model-q4_m.gguf",
#    n_ctx=2048,
#    n_gpu_layers=100, #CPUで実行する場合は削除
#)


history = []

system_message = """
あなたはAIアシスタントです。
"""

def generate_text(message, history):
    temp = ""
    input_prompt = f"{system_message}"
    for interaction in history:
        input_prompt = input_prompt + "\nUSER: " + str(interaction[0]) + "\nASSISTANT: " + str(interaction[1])
    input_prompt = input_prompt + "\nUSER: " + str(message) + "\nASSISTANT: "

    output = llm.create_completion(
        input_prompt,
        temperature=0.7,
        top_p=0.3,
        top_k=40,
        repeat_penalty=1.1,
        max_tokens=1024,
        stop=[
            "ASSISTANT:",
            "USER:",
            "SYSTEM:",
        ],
        stream=True,
    )
    for out in output:
        stream = copy.deepcopy(out)
        temp += stream["choices"][0]["text"]
        yield temp

    history = ["init", input_prompt]


demo = gr.ChatInterface(
    generate_text,
    title="Japanese chatbot using llama-cpp-python",
    description="",
    examples=["日本の四国にある県名を挙げてください。"],
    cache_examples=True,
    retry_btn=None,
    undo_btn="Remove last",
    clear_btn="Clear all",
)
demo.queue(concurrency_count=1, max_size=5)
demo.launch(debug=True, share=True)